In [149]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [150]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\autogluon_type_class_DY"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [151]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\dy\train_data\train_20240619.csv")

In [152]:
dataset.columns

Index(['OBJECTID', 'DLMC', '母质', 'TL', 'YL', 'TS', 'TZ', 'XMin', 'YMin',
       'XMax',
       ...
       'DZ_MAX', 'DZ_RANGE', 'DZ_MEAN', 'DZ_STD', 'DZ_SUM', 'DZ_VARIETY',
       'DZ_MAJORITY', 'DZ_MINORITY', 'DZ_MEDIAN', 'DZ_PCT90'],
      dtype='object', length=403)

In [153]:
dataset.rename(columns={'母质': 'MZ'}, inplace=True)

In [154]:
dataset["TZ_label"] = dataset.TZ.astype("category").cat.codes

In [155]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['MZ'] = dataset['MZ'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [156]:
result = dataset.groupby('TZ_label', observed=True)["TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质中性紫色土', 1: '中层壤质酸性紫色土', 2: '中层壤质黄色石灰土', 3: '中层泥砂质黄壤', 4: '中层泥质黄壤', 5: '中层灰泥质黄壤', 6: '中层灰泥质黄色石灰土', 7: '中层砂泥质黄壤', 8: '中层砂泥黄壤', 9: '中层砾壤质黑色石灰土', 10: '中层硅质酸性粗骨土', 11: '中层红泥质黄壤', 12: '中层黏质黄色石灰土', 13: '厚层泥质黄壤', 14: '厚层灰泥质黄壤', 15: '厚层硅质酸性粗骨土', 16: '厚层红泥质黄壤', 17: '厚层黏质黄色石灰土', 18: '浅石灰泥田', 19: '潮泥田', 20: '石灰泥田', 21: '紫泥田', 22: '腐中层壤质中性紫色土', 23: '腐中层壤质酸性紫色土', 24: '腐中层壤质黄色石灰土', 25: '腐中层泥砂质黄壤', 26: '腐中层泥质黄壤', 27: '腐中层灰泥质黄壤', 28: '腐中层砂泥质黄壤', 29: '腐中层砾壤质黑色石灰土', 30: '腐中层硅质山地草甸土', 31: '腐中层硅质酸性粗骨土', 32: '腐中层硅质黄壤', 33: '腐中层红泥质黄壤', 34: '腐中层黏质黄色石灰土', 35: '腐厚层泥质黄壤', 36: '腐厚层灰泥质黄壤', 37: '腐厚层砂泥质黄壤', 38: '腐厚层硅质酸性粗骨土', 39: '腐厚层硅质黄壤', 40: '腐厚层红泥质黄壤', 41: '腐厚层黏质黄色石灰土', 42: '腐薄层壤质酸性紫色土', 43: '腐薄层壤质黄色石灰土', 44: '腐薄层壤质黑色石灰土', 45: '腐薄层泥砂质黄壤', 46: '腐薄层泥质黄壤', 47: '腐薄层灰泥质黄壤', 48: '腐薄层砂泥质黄壤', 49: '腐薄层砾硅质黄壤', 50: '腐薄层硅质山地灌丛草甸土', 51: '腐薄层硅质山地草甸土', 52: '腐薄层硅质酸性粗骨土', 53: '腐薄层硅质黄壤', 54: '腐薄层黏质黄色石灰土', 55: '薄层壤质酸性紫色土', 56: '薄层壤质黑色石灰土', 57: '薄层泥质黄壤', 58: '薄层灰泥质黄壤', 59: '薄层灰泥质黄色石灰土', 60: '薄层砂泥质黄壤', 61: '薄层砂泥黄壤', 62: '薄层砾

In [157]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20240619.json', 'w') as f:
    json.dump(result, f)

In [158]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DLMC'].value_counts())

In [159]:
dl_df

,count
DLMC,
水田,1861
乔木林地,1101
旱地,740
灌木林地,451
其他草地,135
茶园,76
果园,32
其他林地,30
其他园地,24


In [160]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZ'].value_counts())

In [161]:
mz_df

,count
MZ,
砂岩,1931
碳酸岩,1095
泥(页)岩,487
第四系红粘土,344
砂页岩,251
紫红色砂页岩,155
河流冲积物,82
砾岩,71
第四纪冰川冲积物,65


### 砂岩土种

In [162]:
# 筛选砂岩水稻土数据
sy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '砂岩')]
pd.unique(sy_sdt_data['TZ'])

array(['潮泥田', '黄白粉泥田', '黄青白粉泥田', '黄青砂泥田', '黄浅白粉泥田'], dtype=object)

In [163]:
# 筛砂岩选非自然土数据
sy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '砂岩')]
pd.unique(sy_fzrt_data['TZ']),pd.unique(sy_fzrt_data['TS'])

(array(['薄层硅质黄壤', '薄层砾硅质黄壤'], dtype=object),
 array(['硅质黄壤', '砾硅质黄壤'], dtype=object))

In [165]:
# 筛选自然土数据
sy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '砂岩')]
pd.unique(sy_zrt_data['TZ']),pd.unique(sy_zrt_data['TS'])

(array(['腐厚层硅质黄壤', '腐薄层硅质山地灌丛草甸土', '腐中层硅质黄壤', '腐薄层硅质黄壤', '腐薄层砾硅质黄壤',
        '薄层硅质黄壤', '腐中层硅质山地草甸土', '腐薄层硅质山地草甸土'], dtype=object),
 array(['硅质黄壤', '硅质山地灌丛草甸土', '砾硅质黄壤'], dtype=object))

### 碳酸岩土种

In [166]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '碳酸岩')]
pd.unique(tsy_sdt_data['TZ']),pd.unique(tsy_sdt_data['TS']),

(array(['潮泥田', '石灰泥田', '青石灰泥田', '浅石灰泥田'], dtype=object),
 array(['潮泥田', '石灰泥田', '青灰泥田', '浅灰泥田'], dtype=object))

In [167]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ( dataset['TZ'].str.contains('灰')) & (dataset['MZ'] != '泥(页)岩') &(dataset['MZ'] != '第四系红粘土')]
pd.unique(tsy_fzrt_data['TZ']),pd.unique(tsy_fzrt_data['TS']),

(array(['中层灰泥质黄色石灰土', '薄层灰泥质黄色石灰土', '中层壤质黄色石灰土', '中层砾壤质黑色石灰土',
        '薄层砾壤质黑色石灰土', '薄层壤质黑色石灰土', '薄层灰泥质黄壤', '中层灰泥质黄壤', '厚层灰泥质黄壤'],
       dtype=object),
 array(['灰泥质黄色石灰土', '壤质黄色石灰土', '砾壤质黑色石灰土', '壤质黑色石灰土', '灰泥质黄壤'],
       dtype=object))

In [168]:
# 筛选碳酸岩自然土数据
tsy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & ( dataset['TZ'].str.contains('灰')) & (dataset['MZ'] != '泥(页)岩') &(dataset['MZ'] != '第四系红粘土')]
pd.unique(tsy_zrt_data['TZ']),pd.unique(tsy_zrt_data['TS']),

(array(['腐中层灰泥质黄壤', '腐厚层灰泥质黄壤', '腐薄层灰泥质黄壤', '腐中层壤质黄色石灰土', '腐薄层壤质黑色石灰土',
        '腐薄层壤质黄色石灰土', '腐中层砾壤质黑色石灰土'], dtype=object),
 array(['灰泥质黄壤', '壤质黄色石灰土', '壤质黑色石灰土', '砾壤质黑色石灰土'], dtype=object))

### 第四系红粘土

In [169]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_sdt_data['TZ']),pd.unique(hnt_sdt_data['TS']),

(array(['黄红泥田', '潮泥田', '黄浅红泥田'], dtype=object),
 array(['红泥田', '潮泥田', '浅红泥田'], dtype=object))

In [170]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_fzrt_data['TZ']),pd.unique(hnt_fzrt_data['TS']),

(array(['中层红泥质黄壤', '厚层红泥质黄壤', '薄层黏质黄色石灰土', '厚层黏质黄色石灰土', '中层黏质黄色石灰土'],
       dtype=object),
 array(['红泥质黄壤', '黏质黄色石灰土'], dtype=object))

In [171]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_zrt_data['TZ']),pd.unique(hnt_zrt_data['TS']),

(array(['腐中层黏质黄色石灰土', '腐厚层黏质黄色石灰土', '腐中层红泥质黄壤', '腐厚层红泥质黄壤', '腐薄层黏质黄色石灰土'],
       dtype=object),
 array(['黏质黄色石灰土', '红泥质黄壤'], dtype=object))

### 泥(页)岩

In [172]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_sdt_data['TZ']),pd.unique(nyy_sdt_data['TS']),

(array(['潮泥田', '黄浅鳝泥田', '黄鳝泥田'], dtype=object),
 array(['潮泥田', '浅鳝泥田', '鳝泥田'], dtype=object))

In [173]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_fzrt_data['TZ']),pd.unique(nyy_fzrt_data['TS']),

(array(['薄层黏质黄色石灰土', '中层泥质黄壤', '薄层泥质黄壤', '厚层泥质黄壤'], dtype=object),
 array(['黏质黄色石灰土', '泥质黄壤'], dtype=object))

In [174]:
# 筛选泥(页)岩自然土数据
nyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_zrt_data['TZ']),pd.unique(nyy_zrt_data['TS']),

(array(['腐薄层泥质黄壤', '腐厚层泥质黄壤', '腐薄层黏质黄色石灰土', '腐中层泥质黄壤', '腐厚层黏质黄色石灰土',
        '腐中层黏质黄色石灰土'], dtype=object),
 array(['泥质黄壤', '黏质黄色石灰土'], dtype=object))

### 紫红色砂页岩	

In [232]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZ'] == '紫红色砂页岩') | (dataset['MZ'] == '河流冲积物'))]
pd.unique(zhsyy_sdt_data['TZ']),pd.unique(zhsyy_sdt_data['TS']),

(array(['潮泥田', '紫泥田'], dtype=object), array(['潮泥田', '紫泥田'], dtype=object))

In [176]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '紫红色砂页岩')]
pd.unique(zhsyy_fzrt_data['TZ']),pd.unique(zhsyy_fzrt_data['TS']),

(array(['中层壤质中性紫色土', '中层壤质酸性紫色土', '薄层壤质酸性紫色土'], dtype=object),
 array(['壤质中性紫色土', '壤质酸性紫色土'], dtype=object))

In [177]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '紫红色砂页岩')]
pd.unique(zhsyy_zrt_data['TZ']),pd.unique(zhsyy_zrt_data['TS']),

(array(['腐中层壤质酸性紫色土', '腐薄层壤质酸性紫色土', '腐中层壤质中性紫色土'], dtype=object),
 array(['壤质酸性紫色土', '壤质中性紫色土'], dtype=object))

### 河流冲积物

In [178]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂'])]
pd.unique(hlcjw_sdt_data['TZ']),pd.unique(hlcjw_sdt_data['TS']),

(array(['黄红泥田', '潮泥田', '黄白粉泥田', '石灰泥田', '黄浅鳝泥田', '青石灰泥田', '浅石灰泥田',
        '黄青白粉泥田', '黄青砂泥田', '黄浅白粉泥田', '黄浅红泥田', '紫泥田', '黄鳝泥田', '黄砂泥田',
        '重漂砂泥田'], dtype=object),
 array(['红泥田', '潮泥田', '青白粉泥田', '石灰泥田', '浅鳝泥田', '青灰泥田', '浅灰泥田', '青砂泥田',
        '浅白粉泥田', '浅红泥田', '紫泥田', '鳝泥田', '砂泥田', '漂砂泥田'], dtype=object))

### 砾岩

In [179]:
# 筛选砾岩水稻土数据
ly_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_sdt_data['TZ']),pd.unique(ly_sdt_data['TS']),

(array(['潮泥田', '黄白粉泥田', '黄青白粉泥田', '黄青砂泥田', '黄浅白粉泥田'], dtype=object),
 array(['潮泥田', '青白粉泥田', '青砂泥田', '浅白粉泥田'], dtype=object))

In [180]:
# 筛选砾岩非自然土数据
ly_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_fzrt_data['TZ']),pd.unique(ly_fzrt_data['TS']),

(array(['薄层硅质黄壤', '薄层砾硅质黄壤', '中层硅质酸性粗骨土', '厚层硅质酸性粗骨土'], dtype=object),
 array(['硅质黄壤', '砾硅质黄壤', '硅质酸性粗骨土'], dtype=object))

In [182]:
# 筛选砾岩非自然土数据
ly_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_fzrt_data['TZ']),pd.unique(ly_fzrt_data['TS']),

(array(['腐厚层硅质黄壤', '腐中层硅质黄壤', '腐薄层硅质黄壤', '腐薄层砾硅质黄壤', '薄层硅质黄壤',
        '腐中层硅质酸性粗骨土', '腐厚层硅质酸性粗骨土', '腐薄层硅质酸性粗骨土'], dtype=object),
 array(['硅质黄壤', '砾硅质黄壤', '硅质酸性粗骨土'], dtype=object))

### 第四纪冰川冲积物

In [183]:
# 筛选第四纪冰川冲积物水稻土数据
bccjw_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                     ]
pd.unique(bccjw_sdt_data['TZ']),pd.unique(bccjw_sdt_data['TS']),

(array(['黄红泥田', '潮泥田', '黄白粉泥田', '石灰泥田', '黄浅鳝泥田', '青石灰泥田', '浅石灰泥田',
        '黄青白粉泥田', '黄青砂泥田', '黄浅白粉泥田', '黄浅红泥田', '紫泥田', '黄鳝泥田', '黄砂泥田',
        '重漂砂泥田'], dtype=object),
 array(['红泥田', '潮泥田', '青白粉泥田', '石灰泥田', '浅鳝泥田', '青灰泥田', '浅灰泥田', '青砂泥田',
        '浅白粉泥田', '浅红泥田', '紫泥田', '鳝泥田', '砂泥田', '漂砂泥田'], dtype=object))

In [237]:
# 筛选第四纪冰川冲积物非自然土数据
bccjw_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZ'] == '第四纪冰川冲积物') | (dataset['MZ'] == '砂岩'))]
pd.unique(bccjw_fzrt_data['TZ']),pd.unique(bccjw_fzrt_data['TS']),

(array(['薄层硅质黄壤', '薄层砾硅质黄壤', '中层泥砂质黄壤'], dtype=object),
 array(['硅质黄壤', '砾硅质黄壤', '泥砂质黄壤'], dtype=object))

In [185]:
# 筛选第四纪冰川冲积物自然土数据
bccjw_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '第四纪冰川冲积物')]
pd.unique(bccjw_zrt_data['TZ']),pd.unique(bccjw_zrt_data['TS']),

(array(['腐中层泥砂质黄壤', '腐薄层泥砂质黄壤'], dtype=object), array(['泥砂质黄壤'], dtype=object))

### 砂页岩

In [187]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_sdt_data['TZ']),pd.unique(syy_sdt_data['TS']),

(array(['黄砂泥田', '重漂砂泥田'], dtype=object), array(['砂泥田', '漂砂泥田'], dtype=object))

In [188]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_fzrt_data['TZ']),pd.unique(syy_fzrt_data['TS']),

(array(['中层砂泥质黄壤', '薄层砂泥质黄壤', '中层砂泥黄壤', '薄层砂泥黄壤'], dtype=object),
 array(['砂泥质黄壤', '砂泥黄壤'], dtype=object))

In [189]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_zrt_data['TZ']),pd.unique(syy_zrt_data['TS']),

(array(['腐中层砂泥质黄壤', '腐薄层砂泥质黄壤', '腐厚层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤'], dtype=object))

In [190]:
# 定义要检查的关键字列表
keywords = ['DEM','ndvi', 'PCA_0', 'Slope','Aspect', 'MRVBF','TopographicWetnessIndex', 'Mean',  'PH','PRE','SRA','TMP','VAP','WIND','PlanCurvature']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


160
['DEM_COUNT', 'DEM_AREA', 'DEM_MIN', 'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN', 'DEM_PCT90', 'Aspect_COUNT', 'Aspect_AREA', 'Aspect_MIN', 'Aspect_MAX', 'Aspect_RANGE', 'Aspect_MEAN', 'Aspect_STD', 'Aspect_SUM', 'Aspect_MEDIAN', 'Aspect_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA', 'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE', 'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM', 'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'RelativeSlopePosition_COUNT', 'RelativeSlopePosition_AREA', 'RelativeSlopePosition_MIN', 'RelativeSlopePosition_MAX', 'RelativeSlopePosition_RANGE', 'RelativeSlopePosition_MEAN', 'RelativeSlopePosition_STD', 'RelativeSlopePosition_SUM', 'RelativeSlopePosition_MEDIAN', 'RelativeSlopePosition_PCT90', 'Slope_COUNT', 'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN', 'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'TopographicWetnessIndex_COUNT', 'TopographicWetnessIndex_AR

In [191]:

sdt_features= ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
fzrt_features = ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
zrt_features = ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
target = "TZ_label"

### 砂岩数据集

In [192]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

### 碳酸岩数据集

In [193]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [194]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [195]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [233]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [197]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]

### 砾岩数据集

In [198]:
# 获取sdt数据集
ly_sdt_data = ly_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
ly_fzrt_data = ly_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
ly_zrt_data = ly_zrt_data[zrt_features+[target]]

### 第四纪冰川冲积物数据集

In [238]:
# 获取sdt数据集
bccjw_sdt_data = bccjw_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
bccjw_fzrt_data = bccjw_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
bccjw_zrt_data = bccjw_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [200]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

In [239]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape)
print('砾岩',ly_sdt_data.shape,ly_fzrt_data.shape,ly_zrt_data.shape)
print('第四纪冰川冲积物',bccjw_sdt_data.shape,bccjw_fzrt_data.shape,bccjw_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)


砂岩 (1464, 37) (29, 37) (438, 37)
碳酸岩 (104, 37) (431, 37) (560, 37)
第四系红粘土 (134, 37) (53, 37) (157, 37)
泥页岩 (44, 37) (51, 37) (392, 37)
紫红色砂页岩 (94, 37) (69, 37) (74, 37)
河流冲积物 (1861, 37)
砾岩 (1464, 37) (53, 37) (465, 37)
第四纪冰川冲积物 (1861, 37) (49, 37) (45, 37)
砂页岩 (21, 37) (210, 37) (20, 37)


In [46]:
# # 分割训练集和测试集
# sdt_data_train,sdt_data_test = train_test_split(sdt_data, test_size=0.2, random_state=42)
# fzrt_data_train,fzrt_data_test = train_test_split(fzrt_data, test_size=0.2, random_state=42)
# zrt_data_train,zrt_data_test = train_test_split(zrt_data, test_size=0.2, random_state=42)

# # data = data
# label = "TZ_label"

In [220]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [229]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [235]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
# 砾岩
ly_sdt_predictor = train_model(ly_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='sdt',model_root=model_path) 
ly_fzrt_predictor = train_model(ly_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='fzrt',model_root=model_path) 
ly_zrt_predictor = train_model(ly_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='zrt',model_root=model_path) 
# 第四纪冰川冲积物
bccjw_sdt_predictor = train_model(bccjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='sdt',model_root=model_path) 
bccjw_fzrt_predictor = train_model(bccjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='fzrt',model_root=model_path) 
bccjw_zrt_predictor = train_model(bccjw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='zrt',model_root=model_path) 
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class_DY\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       44.51 GB / 63.81 GB (69.7%)
Disk Space Avail:   474

文件夹已存在


	0.9821	 = Validation score   (f1_weighted)
	0.82s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.971	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.971	 = Validation score   (f1_weighted)
	0.37s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.967	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.967	 = Validation score   (f1_weighted)
	0.39s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.954	 = Validation score   (f1_weighted)
	1.0s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.9821	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total ru

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.982078  f1_weighted       0.003997  0.910347                0.000000           0.092869            2       True          7
1      NeuralNetFastAI   0.982078  f1_weighted       0.003997  0.817479                0.003997           0.817479            1       True          1
2     RandomForestGini   0.970998  f1_weighted       0.042228  0.382453                0.042228           0.382453            1       True          2
3     RandomForestEntr   0.970998  f1_weighted       0.043204  0.372714                0.043204           0.372714            1       True          3
4       ExtraTreesEntr   0.967041  f1_weighted       0.029569  0.390919                0.029569           0.390919            1       True          5
5       ExtraTreesGini   0.967041  f1_

	0.8148	 = Validation score   (f1_weighted)
	0.31s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8148	 = Validation score   (f1_weighted)
	0.3s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8148	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8148	 = Validation score   (f1_weighted)
	0.31s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 0: early stopping
	0.5333	 = Validation score   (f1_weighted)
	0.1s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8148	 = Validation score   (f1_weighted)
	0.12s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 1.0}
	0.8148	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validatio

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.780952  f1_weighted       0.037328  0.520120                0.000000           0.091120            2       True          7
1       ExtraTreesGini   0.600000  f1_weighted       0.028050  0.305629                0.028050           0.305629            1       True          3
2     RandomForestGini   0.600000  f1_weighted       0.029327  0.308485                0.029327           0.308485            1       True          1
3       ExtraTreesEntr   0.600000  f1_weighted       0.031508  0.445951                0.031508           0.445951            1       True          4
4     RandomForestEntr   0.600000  f1_weighted       0.041704  0.317278                0.041704           0.317278            1       True          2
5       NeuralNetTorch   0.450000  f1_

No improvement since epoch 9: early stopping
	0.6476	 = Validation score   (f1_weighted)
	0.85s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.5576	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.5173	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.3968	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.4733	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6152	 = Validation score   (f1_weighted)
	1.02s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 0.667, 'RandomForestGini': 0.333}
	0.6667	 = Validation score   (f1_weighted)
	0.09s	 = Tra

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.666667  f1_weighted       0.036487  1.291185                0.000000           0.087221            2       True          7
1      NeuralNetFastAI   0.647554  f1_weighted       0.007052  0.853955                0.007052           0.853955            1       True          1
2       NeuralNetTorch   0.615181  f1_weighted       0.007001  1.016168                0.007001           1.016168            1       True          6
3     RandomForestGini   0.557644  f1_weighted       0.029434  0.350009                0.029434           0.350009            1       True          2
4     RandomForestEntr   0.517266  f1_weighted       0.028107  0.380946                0.028107           0.380946            1       True          3
5       ExtraTreesEntr   0.473328  f1_

No improvement since epoch 0: early stopping
	0.1667	 = Validation score   (f1_weighted)
	0.11s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	1.0	 = Validation score   (f1_weighted)
	0.32s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	1.0	 = Validation score   (f1_weighted)
	0.34s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	1.0	 = Validation score   (f1_weighted)
	0.32s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6667	 = Validation score   (f1_weighted)
	0.12s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestGini': 1.0}
	1.0	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
A

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   1.000000  f1_weighted       0.030205  0.406182                0.000000           0.085415            2       True          7
1     RandomForestGini   1.000000  f1_weighted       0.030205  0.320766                0.030205           0.320766            1       True          2
2       ExtraTreesEntr   1.000000  f1_weighted       0.032112  0.324711                0.032112           0.324711            1       True          5
3       ExtraTreesGini   1.000000  f1_weighted       0.032864  0.336465                0.032864           0.336465            1       True          4
4     RandomForestEntr   1.000000  f1_weighted       0.033782  0.327526                0.033782           0.327526            1       True          3
5       NeuralNetTorch   0.666667  f1_